In this notebook I'm going to implement the model based on Hu & Bentley's article published in 2000.

This notebook is a proof-of-concept - I'll package the code later.

In [ ]:
import numpy as np
import pandas as pd
import scipy.special
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from baculovirus_vlp import pinf, jmin_jmax, td, Parameters, Simulation

# Supproting Methods

### Infection Probability

Poisson distribution of the infection probability:

$P(t, j) = \frac{exp(-dynMOI) \times dynMOI^j}{j!}$

Thus:

$ln(P(t,j)) = -dynMOI + j \cdot ln(dynMOI) - gammaln(j+1)$

Note that $dynMOI = \alpha * (V1(t) / No(t))$

Here we'll use the same `pinf` method to calculate newly-infection and re-infection, which only differs in $\alpha$.

In [ ]:
pinf(j=5, alpha=0.04, V1=2e6, N=5e5)

### Upper and Lower Bound of Infecting Virus Number j: $j_{MIN}$ and $j_{MAX}$

$j_{MIN}$ and $j_{MAX}$ are determined by dynamic MOI. The rational is to make sure at certain *dynamic MOI* (which is also the $\lambda$ of Poisson distribution), 99% of the probability is within `[jmin, jmax]`.

In [ ]:
print(jmin_jmax(alpha=0.04, V1=2e6, N=5e5))
print(jmin_jmax(alpha=0.04, V1=2e8, N=5e5))
print(jmin_jmax(alpha=0.04, V1=2e10, N=5e5))

### Infected cell death time cutoff $t_d$

Model parameter $t_d$ (time after infection cell viability starts to drop quickly (h)) is a function of initial MOI. This is a property of `Simulation`.

In [ ]:
print(td(V1=1e6, No=2e6))    # MOI < 1
print(td(V1=8e6, No=2e6))    # 1 <= MOI <= 20
print(td(V1=5e7, No=2e6))    # MOI > 20

### Virus Synthesis as a Function of j

When calculating virus production, the more virus infecting a cell, the higher the rate of synthesis. But *the existance of viral chromosomes inside the cells increases the metabolic burden*, thus the authors used a logarithmic function to describe the dependence of virus synthesis and j.

This is a method of class `Parameters` since its value depends on the maximum number of virus that may affect the virus and protein synthesis ($v_{max}$).

### Onset Time of Virion (and Protein) Synthesis

The onset time of synthesis $\tau$ has a capped linear relationship with j. The higher the j, the faster the synthesis. $\tau$ is the time so $\tau_{high}$ is bigger (slower) than $\tau_{low}$. This method is also a method of `Parameters`.

# `Parameters` dataclass

Use a dataclass to store the parameters used in the model. This way the nomenclature is consistent and clear.

In [ ]:
params = Parameters()

In [ ]:
params.__dict__

In [ ]:
params.fj(j=10)

In [ ]:
params.tau(j=30)

# Class `Simulation`

Implement the model by a `Simulation` class. 

Attributes:
* `Parameters`
* Simulation settings: running time, time frame
* Starting conditions: MOI, cell numbers, concentrations
* Simulation run results
    * Cells: infected, non-infected, dead
    * Virus: production, initial-adsorption, re-infection
    * Substrate

Methods are the steps of updating results in each time frame:
* Update substrate
* Infection
* Update cell death
* Re-infection
* Virus production

In [ ]:
sim = Simulation(
    params=params, 
    MOI=1, 
    timestep=1, 
    endtime=10, 
    No_init=2e6, 
    S_init=13
)

In [ ]:
sim

In [ ]:
sim.step()

In [ ]:
sim.to_df()

In [ ]:
sim.run()

In [ ]:
sim.to_df()

In [ ]:
sim.stats

# Sanity Test

The authors didn't provide experimental data for validation. But we can validate their conclusions:
1. (consistent) Fig1b, infection percentage vs. time
2. (consistent) Fig2, Baculovirus conc (vIBD-7) vs. time
3. (different) Fig3a, max virus yield vs. combination of MOI and ICD (No_init).
4. (consistent) Fig4b, viability vs. time.
5. (consistent & different) j distribution.

### Infection Percentage vs. Time

Infection percentage is calculated as infected cell in total cells. MOIs were 0.1, 1, 10. Starting cell concentration was 1.3e6. The experiment ran for 80 hours.

In [ ]:
params = Parameters()

In [ ]:
sims = dict.fromkeys([0.1, 1, 10])

for moi in sims.keys():
    sim = Simulation(
        params=params, 
        MOI=moi, 
        timestep=1, 
        endtime=100, 
        No_init=1.3e6, 
        S_init=13
    )
    sim.run()
    sims[moi] = sim.to_df()

In [ ]:
pct_t = pd.DataFrame(data={
    moi: (sims[moi]["Ninf"] / (sims[moi]["Ninf"] + sims[moi]["No"]))
    for moi in sims.keys()
})

In [ ]:
plt.plot(pct_t[:80])
plt.legend(pct_t.columns)

This figure is basically the same as fig1b.

### Baculovirus Concentration vs. Time

Fig2 has the same simulation settings as fig1.

In [ ]:
v1_t = pd.DataFrame(data={
    moi: sims[moi]["V1"] for moi in sims.keys()
})
fig, axs = plt.subplots(1, 2, figsize=(8, 4))
axs[0].plot(v1_t[:30].apply(np.log10))
axs[0].legend(v1_t.columns)
axs[1].plot(v1_t)
axs[1].legend(v1_t.columns)

The trend and order are the same but the curve shapes are a little different.

### Max virus yield vs. combination of MOI and ICD

Fig3a: maximum virus titer predicted at different MOI and infecting cell densities (ICD).

* MOIs: 0.1, 1, 5, 10, 20
* No_init (10^6 cells/ml): 1.3, 2.2, 3.7, 7.7

In [ ]:
max_vir_df = pd.DataFrame(
    columns=["max_virus_titer"], 
    index=pd.MultiIndex.from_product(
        [(0.1, 1, 5, 10, 20), (1.3e6, 2.2e6, 3.7e6, 7.7e6)],
        names=["MOI", "ICD"]
    )
)
for moi in (0.1, 1, 5, 10, 20):
    for No_init in (1.3e6, 2.2e6, 3.7e6, 7.7e6):
        sim = Simulation(
            params=params, 
            MOI=moi, 
            timestep=1, 
            endtime=100, 
            No_init=No_init, 
            S_init=13
        )
        sim.run()
        max_vir_df.loc[(moi, No_init), "max_virus_titer"] = max(sim.results["V1"])

Change the max titer unit to 1e8.

In [ ]:
max_vir_dfplot = max_vir_df.map(lambda v: round(v/1e8, 1)).unstack()

In [ ]:
sns.heatmap(
    max_vir_dfplot, 
    annot=True,
    xticklabels=max_vir_dfplot.columns.levels[1], 
    yticklabels=max_vir_dfplot.index,
    cmap="Blues"
)
plt.xlabel("ICD")

This one is different from fig3a. Especially the ICD=3.7e6 MOI=20 simulation.

### Viability vs. Time

Compare simulations with fig4b. The MOIs are 01, 1, 10. ICD is 1.3e6. 

The authors didn't mention how they simulated viability. I used a rolling sum with sigmoid weights (linear weights also works) on dead cells of each step as the dead cell concentration.

In [ ]:
sims = dict.fromkeys([0.1, 1, 10])

for moi in sims.keys():
    sim = Simulation(
        params=params, 
        MOI=moi, 
        timestep=1, 
        endtime=100, 
        No_init=1.3e6, 
        S_init=13
    )
    sim.run()
    sims[moi] = sim.to_df()

In [ ]:
def linear_weights(window_size):
    """Create linearly increasing weights"""
    ladder = np.arange(1, window_size + 1)
    return ladder / window_size

def sigmoid_weights(window_size, scale=4):
    """Using scipy's sigmoid (expit) function"""
    x = np.linspace(-scale, scale, window_size)
    weights = scipy.special.expit(x)  # equivalent to 1 / (1 + exp(-x))
    return weights

def weighted_sum(x, weight_func: callable):
    weights = weight_func(len(x))
    return np.sum(x * weights)

In [ ]:
viab_df = pd.DataFrame(columns=sims.keys())

for moi, df in sims.items():
    d_conc = (
        df.dNd
        .rolling(window=100, min_periods=1)
        .apply(
            weighted_sum, 
            # args=(linear_weights, )
            args=(sigmoid_weights, )
        )
    )
    d_pct = d_conc / (df.No + df.Ninf + d_conc)
    viab_df[moi] = 1 - d_pct

plt.plot(viab_df)
plt.ylim(0, 1)
plt.legend(viab_df.columns)

By tuning the hyperparameters of the dead cell calculation, this figure is quite similar to fig4b. They are still slightly different, especially at later stages (the authors simulation retained more dead cells).

### j Distribution

The j distribution in this article is to count all initial infection events (not the re-infection). The result is a mapping of j to N, from the first step until a 100% infection. To count the j distribution I added a `stats` attribute in the `Simulation` object and implement the counting of j distribution in the `_step_newinf` method.

Data in fig6a were simulated at MOIs 0.1, 1, 2.5, 5, 10, 20, 40 and ICD 1.3e6.

In [ ]:
sims = dict.fromkeys([0.1, 1, 2.5, 5, 10, 20, 40])

for moi in sims.keys():
    sim = Simulation(
        params=params, 
        MOI=moi, 
        timestep=1, 
        endtime=100, 
        No_init=1.3e6, 
        S_init=13
    )
    sim.run()
    sims[moi] = sim

In [ ]:
j_distr_freq = pd.DataFrame(
    {moi: sim.stats["j_distr_newinf"] for moi, sim in sims.items()}
).sort_index()
j_distr_log10 = j_distr_freq.map(lambda v: 0 if v<1 else np.log10(v))
j_distr_freq /= j_distr_freq.sum()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].plot(j_distr_log10[[0.1, 1, 10, 20, 40]])
axs[0].legend([0.1, 1, 10, 20, 40])
axs[0].set_ylim(1, 7)
axs[0].set_yticks(ticks=range(1, 8), labels=[f"1E+{i}" for i in range(1, 8)])
axs[0].set_xlim(1, 300)
axs[0].set_title("infected cell conc. by j")

axs[1].plot(j_distr_freq[[0.1, 10, 20, 40]])
axs[1].legend([0.1, 10, 20, 40])
axs[1].set_ylim(0, 1)
axs[1].set_xlim(1, 7)
axs[1].set_title("frac of infected cells by j")

axs[2].plot(j_distr_log10[[0.1, 1, 2.5, 5]])
axs[2].legend([0.1, 1, 2.5, 5])
axs[2].set_ylim(1, 7)
axs[2].set_yticks(ticks=range(1, 8), labels=[f"1E+{i}" for i in range(1, 8)])
axs[2].set_xlim(1, 10)
axs[2].set_title("infected cell conc. by j")

Fig6a and panel 1 have similar trend for MOI 40, 20, and 10. However, MOI 0.1 and 1 are different. MOI 20 has similar shape but different peak position.  
Fig6b and panel 2 are quite similar.  
Fig6c and panel 3 are different.

# Remaining issues

* Whether to round the cell / virus
* Deal with the close to 0 numbers
* Visualization module